## ReadMe 

- Install interactive slider package in a proper environment: conda install ipywidgets matplotlib ipython

In [1]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np 
from matplotlib.colors import LinearSegmentedColormap
import math
import pandas as pd
from IPython.display import display
import ipywidgets as widgets
from scipy.interpolate import griddata

In [38]:
def print_base_stats(I, m_w_product, m_oxygen_usage, m_air_tot, FL):
    print("\n---------------------------------------")
    print("Current:  \t \t", np.round(I, 2), "\tA")
    print("Product Water:  \t", np.round(m_w_product*1000, 2), "\tg/s")
    print("Oxygen Consumption:  \t", np.round(m_oxygen_usage*1000, 2), "\tg/s")
    print("Fuel Consumption:  \t", np.round((m_w_product-m_oxygen_usage)*1000, 2), "\tg/s")
    print("Intake Air:  \t \t", np.round(m_air_tot*1000, 1), "\tg/s")
    print("FL: \t \t \t", FL, "\n---------------------------------------")
    
def p_at_FL(FL):
    
    L = 0.00976 # Temperature Lapse Rate (K/m)
    g = 9.81 # m/s2
    p0 = 101325 # Sea level standard pressure (Pa)
    cp = 1004.69 # J/(kg·K) 
    T0 = 288.16 # Sea level standard temperature (K)
    M = 0.02897 # Molar Mass of dry air kg/mol 
    R0 = 8.314 # J/(mol·K) 
    
    h = FL*100/3.281 # Altitude in m
    
    p_at_alt = p0 * (1 - g * h / (cp * T0)) ** (cp * M / R0)
    p_bar = p_at_alt/101325
    
    return p_bar

def T_at_FL(FL):
    
    # Temperature is assumed to decrease linearly with altitude in the troposphere
    h = FL*100/3.281 # Altitude in m
    T = 20 - h/1000 * 6.5
    
    return T
    
def water_saturation_pressure(T):
    A = 8.07131
    B = 1730.63
    C = 233.426

    log10_P = A - (B / (T + C))
    P_mmHg = 10 ** log10_P
    P_bars = P_mmHg * 0.00133322

    return P_bars

class State:
    def __init__(self, name, mass_flow_W, mass_flow_O, mass_flow_N, T, p):
        self.name = name
        self.mW = mass_flow_W     # kg/s
        self.mO = mass_flow_O     # kg/s
        self.mN = mass_flow_N     # kg/s
        self.T = T                # T in celsius
        self.p = p                # p in bar
        self.p_sat = -1e2         # p in bar
        self.RH = -1e2            # 0->1
        self.p_w = -1e2           # in bar
        self.O_ratio = -1e2       # 0->1
        self.cp = -1e2            # kJ/kgK
        self.m_tot = -1e2         # kg/s
        
    def update(self):
        self.p_sat = water_saturation_pressure(self.T)
        a = self.mW / (0.622 * (self.mO + self.mN))
        self.p_w = a/(1+a) * self.p
        self.RH = self.p_w / self.p_sat
        self.O_ratio = self.mO / (self.mO + self.mW + self.mN)
        self.m_tot = self.mW + self.mO + self.mN
        
        # cp is calculated as from mixture of water vapour + air: cp of vapour assumed constant
        self.cp = ( 1.04 * (self.mO + self.mN) + 1.95 * self.mW ) / (self.mO + self.mN + self.mW)
        
class Compressor_State:
    def __init__(self, name, W, DT, n, surge):
        self.name = name
        self.W = W                 # kW
        self.DT = DT               # C
        self.n = n                 # RPM
        self.surge = surge         # kg/s
        
def compute_state_stack_entrance(I, N_cell, λ):
    
    # assumption for now: p and T at stack entrance constant -> Refine with data!
    p = 2              # bara
    T = 70             # C
    RH = .65
    
    return p, T, RH

def compute_mw_from_RH(m_a, T, p, RH):
    
    p_sat = water_saturation_pressure(T)
    m_w = 0.622 * RH * p_sat / (p - RH * p_sat) * m_a

    return m_w

## Compressor Map

- Compressor used is VSEC15 with data from: https://h2flygmbh.sharepoint.com/:x:/r/sites/H175Sub-Program/_layouts/15/Doc.aspx?sourcedoc=%7BD3AA090C-FEB9-47CF-A604-2F7A48C36CBB%7D&file=Compressor%20map%20data%20for%20H2Fly-%2020240321.xlsx&action=default&mobileredirect=true
- Cubic interpolation using the griddata package
- Surge line is set with manual parameters for the moment: $PR = C \cdot \dot{m}^{k} + 1$
- The compressor interpolation here is very quick-and-dirty. Stefan Notter has implemented a very good compressor map previously which could be incorporated here if needed.

In [54]:

C_surge = 0.0045
k_surge = 1.25

def convert_comma_to_dot(value):
    return float(value.replace(',', '.'))
data = np.genfromtxt('VSEC15_data_with_efficiency.csv', delimiter=';', skip_header=1, dtype=None, 
    encoding=None, converters={1: convert_comma_to_dot, 2: convert_comma_to_dot})

n_data = data['f0'] 
m_corr_data = data['f2'] 
PR_data = data['f1']
η_data = data['f3']
η_data = np.char.replace(η_data, ',', '.')
η_data = np.array(η_data, dtype=float)

def compressor_map(id1, id2, m_corr1, pr1, m_corr2, pr2, graphic, bypass, method='cubic'):
    if(graphic):
        fig, axs = plt.subplots(1, 2, figsize=(12, 5))

    # surge line
    m_surge = np.arange(20, 150, 1)
    pr_surge = C_surge * m_surge**k_surge + 1
    #if(graphic):
        #axs[0].plot(m_surge, pr_surge, color='black', linewidth=1, zorder=0)

    # Check if input compressor operating points are above surge line
    surge1 = False
    surge2 = False
    surge1 = pr1 >= C_surge * m_corr1**k_surge + 1
    surge2 = pr2 >= C_surge * m_corr2**k_surge + 1

    # RPM Plot and Interpolation

    points = np.array([m_corr_data, PR_data]).T
    n1_interp = griddata(points, n_data, (m_corr1, pr1), method=method)
    n2_interp = griddata(points, n_data, (m_corr2, pr2), method=method)

    # Plot the data and operating points 1 and 2
    if(graphic):
        scatter = axs[0].scatter(m_corr_data, PR_data, c=n_data, cmap='viridis')
        cbar = plt.colorbar(scatter, ax=axs[0], label='n (kRPM)')

        if(bypass == False):
            axs[0].scatter(m_corr1, pr1, color='red')
            axs[0].annotate(id1, (m_corr1, pr1), textcoords="offset points", xytext=(5,7), ha='center', color='red', fontsize=11)
            cbar.ax.hlines(n1_interp, *cbar.ax.get_xlim(), color='red', linewidth=3)

        axs[0].scatter(m_corr2, pr2, color='red')
        axs[0].annotate(id2, (m_corr2, pr2), textcoords="offset points", xytext=(5,-15), ha='center', color='red', fontsize=11)
        cbar.ax.hlines(n2_interp, *cbar.ax.get_xlim(), color='red', linewidth=3)

        axs[0].set_xlabel('Corrected Mass Flow (g/s)')
        axs[0].set_ylabel('Pressure Ratio')
        axs[0].set_title('VSEC15 Compressor RPM')

    # Efficiency Plot and Interpolation
    X, Y = np.meshgrid(m_corr_data, PR_data)
    points = np.array([m_corr_data, PR_data]).T  # Points from your original data
    Z = griddata(points, η_data, (X, Y), method='linear')  # Interpolate η_data over the meshgrid

    η1_interp = griddata(points, η_data, (m_corr1, pr1), method='linear')
    η2_interp = griddata(points, η_data, (m_corr2, pr2), method='linear')

    if(graphic):
        contour = axs[1].contourf(X, Y, Z, levels=30, cmap='viridis')
        cbar = plt.colorbar(contour, ax=axs[1], label='η')

        if(bypass == False):
            axs[1].scatter(m_corr1, pr1, color='red')
            axs[1].annotate(id1, (m_corr1, pr1), textcoords="offset points", xytext=(5,7), ha='center', color='red', fontsize=11)
            cbar.ax.hlines(η1_interp, *cbar.ax.get_xlim(), color='red', linewidth=3)
        
        axs[1].scatter(m_corr2, pr2, color='red')
        axs[1].annotate(id2, (m_corr2, pr2), textcoords="offset points", xytext=(5,-15), ha='center', color='red', fontsize=11)
        cbar.ax.hlines(η2_interp, *cbar.ax.get_xlim(), color='red', linewidth=3)

        axs[1].set_xlabel('Corrected Mass Flow (g/s)')
        axs[1].set_ylabel('Pressure Ratio')
        axs[1].set_title('VSEC15 Compressor Efficiency')

    return n1_interp, n2_interp, η1_interp, η2_interp, surge1, surge2

# Test
# n1, n2, η1_interp, η2_interp, surge1, surge2 = compressor_map(id1=1, id2=2, m_corr1 = 140, pr1 = 2.6, m_corr2 = 150, pr2 = 2.3, graphic=True)
# print(n1, n2, η1_interp, η2_interp, surge1, surge2)

### Definition of Valve, HEX and Humidifier Pressure Losses and Compressor Efficiencies

In [67]:
def pressure_losses(m_tot, T, p):
    R_air = 287.1      # J/kgK
    V_tot = m_tot * R_air * (T+273.15) / (p*1e5)
    V_ref = 0.12 * R_air * (T+273.15) / (p*1e5)

    dp_c01_valve = 0.1 *  V_tot / V_ref     # bar
    dp_hex = 0.1 *  V_tot / V_ref             # bar
    dp_hum = 0.2 *  V_tot / V_ref          # bar

    return dp_c01_valve, dp_hex, dp_hum

def evaluate_compressor(m_tot, c_p, p_out, p_in, T_in):
    
    T_in += 273.15        # since T is passed on in celsius
    T_ref = 20 + 273.15   # Reference Temperature K
    
    p_ref = 1             # bar

    m_corr = (p_in / p_ref) * np.sqrt(T_in / T_ref) * m_tot 

    n, n2_notused, η, η2_notused, surge, surge_notused = compressor_map(1, 2, m_corr*1000, p_out/p_in, 0, 0, False, False)
    
    W = m_tot * c_p * T_in / η * ((p_out/p_in)**0.2857 - 1)
    DT = T_in / η * ((p_out/p_in)**0.2857 - 1)   
    return surge, n, DT, W

## Computation of States 1-5

Missing

- Temperature Drop over Humidifier
- Actual Temperature Drop over HX
- Exact Pressure Losses Based on Real-World Data
- Mapping of p, T at stack inlet from Stack Current
- pressure loss over water separators
- Isentropic cooling over compressor bypass valve is ignored. T2 = T1 if bypass enabled

Assumptions 

- Pressure in State 2 (between compressors) set to have be adjustable with a slider where $\frac{PR_2}{PR_2}$ is the fraction of the pressure ratio at the second compressor (usually larger) and the first compressor.

$\frac{PR_2}{PR_2} = \frac{p_3 p_1}{p_2 ^2} $ which means $ p_2 = \sqrt{\frac{p_1 \cdot p_3 }{PR_2 / PR_1}}$

- RH at ambient always 70%, this is hardcoded!

In [75]:
RH_ambient = 0.70

def compute_states(λ, I, FL, N_cell, m_w_product, m_oxygen_usage, PR_ratio, C1_bypass, T_inlet, p_inlet, RH_inlet):
    m_a = λ * m_oxygen_usage + 0.77/0.23 * λ * m_oxygen_usage
        
    state1 = State("(1) C1 Inlet", 0, λ * m_oxygen_usage, 0.77/0.23 * λ * m_oxygen_usage, T_at_FL(FL), p_at_FL(FL))
    
    state1.mW = compute_mw_from_RH(m_a, state1.T, state1.p, RH_ambient)

    
    # p5, T5, RH5 = compute_state_stack_entrance(I, N_cell, λ) not used for the moment

    p5, T5, RH5 = p_inlet, T_inlet, RH_inlet/100
    m_w5 = compute_mw_from_RH(m_a, T5, p5, RH5)
    
    state5 = State("(5) Stack Inlet", m_w5, λ * m_oxygen_usage, 0.77/0.23 * λ * m_oxygen_usage, T5, p5)
    
    dp_c01_valve, dp_hex, dp_hum = pressure_losses(m_a, state5.T, state5.p)
    
    state4 = State("(4) Hum Inlet", state1.mW, λ * m_oxygen_usage, 0.77/0.23 * λ * m_oxygen_usage, T5 - 2, state5.p + dp_hum)
    
    state3 = State("(3) HEX Inlet", state1.mW, λ * m_oxygen_usage, 0.77/0.23 * λ * m_oxygen_usage, T5, state4.p + dp_hex)
        
    state2 = State("(2) C2 Inlet", state1.mW, λ * m_oxygen_usage, 0.77/0.23 * λ * m_oxygen_usage, 0, 0)

    compressor_state_1 = Compressor_State("Compressor Stage 1", 0, 0, 0, False)
    
     # bypass of first compressor causes a pressure loss over bypass valve
    if(C1_bypass==True):
        dp_c01_valve, dp_hex, dp_hum = pressure_losses(m_a, state1.T, state1.p)
        state2.p = state1.p - dp_c01_valve
        # isentropic cooling ignored for the time being
        state2.T = state1.T

    # set p2 to match C1 and C2 compression ratios
    elif(C1_bypass == False):
        state2.p = np.sqrt(state3.p * state1.p / PR_ratio)        
        state1.update()
        surge1, n1, DT1, W1 = evaluate_compressor(state1.m_tot, state1.cp, state2.p, state1.p, state1.T)
        compressor_state_1 = Compressor_State("Compressor Stage 1", W1, DT1, n1, surge1)
        state2.T = state1.T + DT1

    states = [state1, state2, state3, state4, state5] 
    for state in states:
        state.update()
    
    surge2, n2, DT2, W2 = evaluate_compressor(state2.m_tot, state2.cp, state3.p, state2.p, state2.T)
    compressor_state_2 = Compressor_State("Compressor Stage 2", W2, DT2, n2, surge2)
    
    compressor_states = [compressor_state_1, compressor_state_2] 

    state3.T = state2.T + DT2
    state3.update()

    T_ref = 20 + 273.15   # Reference Temperature K
    p_ref = 1             # bar

    m_corr1 = (state1.p / p_ref) * np.sqrt((state1.T + 273.15) / T_ref) * state1.m_tot
    m_corr2 = (state2.p / p_ref) * np.sqrt((state2.T  + 273.15)/ T_ref) * state2.m_tot 

    # Plotting of compressor states
    compressor_map(1, 2, m_corr1*1000, state2.p/state1.p, m_corr2*1000, state3.p/state2.p, True, C1_bypass)
    
    return states, compressor_states

In [76]:
# Define parameter adjustment using sliders, dropdowns, etc.
λ_slider = widgets.FloatSlider(value=1.8, min=1.0, max=3.0, step=0.1, description="λ")
FL_slider = widgets.IntSlider(value=0, min=0, max=350, step=10, description="FL")
T_slider = widgets.IntSlider(value=70, min=20, max=100, step=2, description=r"T St. In. (C)")
p_slider = widgets.FloatSlider(value=2, min=1, max=3.5, step=0.1, description=r"p St. In. (bara)")
RH_slider = widgets.IntSlider(value=65, min=0, max=100, step=5, description=r"RH St. In. (%)")
I_slider = widgets.IntSlider(value=450, min=0, max=700, step=5, description=r"I (A)")
N_cell_slider = widgets.IntSlider(value=455, min=400, max=700, step=5, description="Stack Cells")
PR_ratio_slider = widgets.FloatLogSlider(
    value=1,
    base=10,
    min=-1,  # Corresponds to 10^(-1) = 0.1
    max=1,   # Corresponds to 10^(1) = 10
    step=0.01,
    description=r"$\frac{PR_2}{PR_1}$",
    readout_format='.1f'
)
C1_bypass_slider = widgets.Checkbox(value=True, description="Compressor 1 Bypass")


def update(λ, I, FL, N_cell, PR_ratio, C1_bypass, T_inlet, p_inlet, RH_inlet):

    m_w_product = 9.336 * 1e-8 * I * N_cell
    m_oxygen_usage = 0.5 * 1.66 * 1e-7 * I * N_cell
    m_air_tot = λ * (1/0.23) * m_oxygen_usage 

    
    print_base_stats(I, m_w_product, m_oxygen_usage, m_air_tot, FL)
    
    states, compressor_states = compute_states(λ, I, FL, N_cell, m_w_product, m_oxygen_usage, PR_ratio, C1_bypass, T_inlet, p_inlet, RH_inlet)
            
    img = mpimg.imread('diagram.png')
    fig, ax = plt.subplots(figsize=(12, 12), dpi=200)  # Adjust figsize and dpi for bigger and higher resolution
    ax.axis('off')
    ax.imshow(img)
    
    posx = [0.075, 0.29, 0.42, 0.61, 0.934, 0.8, 0.9]
    posy = [0.15, 0.15, 0.15, 0.15, 0.622, 0.15, 0.15]

    posx_compressor = [0.075, 0.29]
    posy_compressor = [1, 1]

    warning_compressor=False
    warning_temp=False
    if(states[2].T >= 215): 
        print(" Potential Overtemperature at Compressor Outlet. Check States.\n\t - Check FL \n\t - Check Compressor Bypass \n\t - Adjust PR2 / PR1 \n---------------------------------------")
        warning_temp = True
        
    for i in range(len(states)): 
        col = "green"
        if math.isnan(states[i].T) or math.isnan(states[i].p):
            if(warning_compressor==False): print(" Potential Surge or Stall. Check States.\n\t - Check FL \n\t - Check Compressor Bypass \n\t - Adjust PR2 / PR1 \n---------------------------------------")
            col = "red"
            warning_compressor=True
        if(warning_temp == True and i==2): col="red"
        ax.annotate(states[i].name, xy=(posx[i], posy[i]), xycoords='axes fraction', color=col, fontsize=9, ha='center', va='center')
        ax.annotate(f'T = {np.round(states[i].T, 1)} C', xy=(posx[i]-0.04, posy[i] - 0.04), xycoords='axes fraction', color=col, fontsize=7, ha='left', va='center')   
        ax.annotate(f'p = {np.round(states[i].p, 2)} bar', xy=(posx[i]-0.04, posy[i] - 0.065), xycoords='axes fraction', color=col, fontsize=7, ha='left', va='center')   
        ax.annotate(f'm_a = {np.round((states[i].mO + states[i].mN)*1000, 1)} g/s', xy=(posx[i]-0.04, posy[i] - 0.09), xycoords='axes fraction', color=col, fontsize=7, ha='left', va='center')   
        ax.annotate(f'RH = {np.round((states[i].RH)*100, 1)} %', xy=(posx[i]-0.04, posy[i] - 0.115), xycoords='axes fraction', color=col, fontsize=7, ha='left', va='center')   

    for i in range(len(compressor_states)):
        col="green"
        if(warning_compressor==True): col="red"
        ax.annotate(compressor_states[i].name, xy=(posx_compressor[i], posy_compressor[i]), xycoords='axes fraction', color=col, fontsize=9, ha='center', va='center')   
        ax.annotate(f'n = {np.round(compressor_states[i].n, 1)}k RPM', xy=(posx_compressor[i]-0.04, posy_compressor[i] - 0.04), xycoords='axes fraction', color=col, fontsize=7, ha='left', va='center')   
        ax.annotate(f'P = {np.round(compressor_states[i].W, 1)} kW', xy=(posx_compressor[i]-0.04, posy_compressor[i] - 0.065), xycoords='axes fraction', color=col, fontsize=7, ha='left', va='center')   
    plt.show()


ui = widgets.VBox([λ_slider, FL_slider, I_slider, T_slider, p_slider, RH_slider, N_cell_slider, PR_ratio_slider, C1_bypass_slider])
out = widgets.interactive_output(update, {
    'λ': λ_slider,
    'FL': FL_slider,
    'I': I_slider,
    'T_inlet': T_slider, 
    'p_inlet': p_slider, 
    'RH_inlet': RH_slider,
    'N_cell': N_cell_slider,
    'PR_ratio': PR_ratio_slider,
    'C1_bypass': C1_bypass_slider
})

display(ui, out)



Output()